In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class WaterCounter:

    #keep track of all water counters
    water_counters = []

    #price per cubic meter
    price = 1

    def __init__(self, flat, month, current, previous):
        self.flat = flat
        self.month = month
        self.current = current
        self.previous = previous
        WaterCounter.water_counters.append(self)
    
    @classmethod
    def water_per_month(cls, month):
        df_list = []
        for counter in cls.water_counters:
            if counter.month == month:
                df_list.append({'flat': counter.flat, 'water': counter.current - counter.previous})
        return pd.DataFrame(df_list)
    
    @classmethod
    def water_per_quarters(cls):
        df_list = []
        sums = {}
        for counter in cls.water_counters:
            if counter.month in [1,2,3]:

                if counter.flat in sums:
                    sums[counter.flat][0] += counter.current - counter.previous
                else:
                    sums[counter.flat] = [counter.current - counter.previous, 0, 0, 0]
            elif counter.month in [4,5,6]:
                if counter.flat in sums:
                    sums[counter.flat][1] += counter.current - counter.previous
                else:
                    sums[counter.flat] = [0, counter.current - counter.previous, 0, 0]
            elif counter.flat in [7,8,9]:
                if counter.flat in sums:
                    sums[counter.flat][2] += counter.current - counter.previous
                else:
                    sums[counter.flat] = [0, 0, counter.current - counter.previous, 0]
            else:
                if counter.flat in sums:
                    sums[counter.flat][3] += counter.current - counter.previous
                else:
                    sums[counter.flat] = [0, 0, 0, counter.current - counter.previous]
        
        for flat in sums:
            df_list.append({'flat': flat, 'Q1': sums[flat][0], 'Q2': sums[flat][1], 'Q3': sums[flat][2], 'Q4': sums[flat][3]})
        
        df = pd.DataFrame(df_list)
        df.set_index('flat', inplace=True)

        return df
    
    @classmethod
    def all_water_per_month(cls):
        df_list = [{'month': i, "water":0} for i in range(1,13)]
        for counter in cls.water_counters:
            df_list[counter.month-1]['water'] += counter.current - counter.previous
        
        df = pd.DataFrame(df_list)
        df.set_index('month', inplace=True)
        return df
            
    
            
           

In [67]:
df_water = pd.read_csv('water.csv')
df_water

,flat,month,current,previous
0,1,1,120,50
1,2,1,100,40
2,3,2,100,4
3,4,2,70,50
4,5,3,100,50
5,6,3,180,50
6,7,4,100,50
7,8,4,100,50
8,9,5,90,50
9,10,5,100,50


In [68]:
#check if there current is greater than previous
df_water[df_water['current'] < df_water['previous']]

,flat,month,current,previous


In [97]:
def count_diff(x):
    return (x['current'] - x['previous'])*WaterCounter.price

In [70]:
#a
def water_per_month(month, df_water):
    df_water['water'] = df_water.apply(count_diff, axis=1)
    return df_water[df_water['month'] == month][['flat', 'water']].reset_index(drop=True)

In [71]:
water_per_month(2, df_water)

,flat,water
0,3,96
1,4,20
2,1,150


In [72]:
#b
def sum_of_water(df_water):
    df_water['water'] = df_water.apply(count_diff, axis=1)
    return df_water["water"].sum()

In [73]:
sum_of_water(df_water)

846

In [74]:
#c
def quartiles_sum(df_water):
    df_water['water'] = df_water.apply(count_diff, axis=1)
    df_temp = pd.DataFrame(columns=['Q1', 'Q2', 'Q3', 'Q4'])
    df_temp["Q1"] = df_water[df_water['month'].isin([1,2,3])]['water'].reset_index(drop=True)
    df_temp["Q2"] = df_water[df_water['month'].isin([4,5,6])]['water'].reset_index(drop=True)
    df_temp["Q3"] = df_water[df_water['month'].isin([7,8,9])]['water'].reset_index(drop=True)
    df_temp["Q4"] = df_water[df_water['month'].isin([10,11,12])]['water'].reset_index(drop=True)

    return df_temp.sum()

In [75]:
quartiles_sum(df_water)

Q1    576.0
Q2    220.0
Q3     50.0
Q4      0.0
dtype: float64

In [96]:
df_water.groupby('flat').apply(lambda x: x['month'][x['water'].idxmax()])

flat
1     2
2     1
3     2
4     2
5     3
6     3
7     4
8     4
9     5
10    5
11    6
12    6
dtype: int64

In [76]:
#d
def max_water_month_for_flat(df_water):
    df_water['water'] = df_water.apply(count_diff, axis=1)
    df_temp = pd.DataFrame(columns=['flat', 'month', 'water'])
    gb = df_water.groupby('flat')
    df_temp["flat"] = df_water['flat'].unique()
    df_temp["month"] = gb.apply(lambda x: x['month'][x['water'].idxmax()]).reset_index(drop=True)
    df_temp["water"] = gb.apply(lambda x: x['water'].max()).reset_index(drop=True) 

    return df_temp.reset_index(drop=True)

In [77]:
max_water_month_for_flat(df_water)

,flat,month,water
0,1,2,150
1,2,1,60
2,3,2,96
3,4,2,20
4,5,3,50
5,6,3,130
6,7,4,50
7,8,4,50
8,9,5,40
9,10,5,50
